<a href="https://colab.research.google.com/github/RyanChen12035/w266-NLP/blob/main/w266_final_project_model3_pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pydot --quiet
!pip install tensorflow-datasets --quiet
!pip install transformers --quiet

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-efc50073-ef7c-b260-654a-62da1bed70a7)


In [4]:
import os
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Dropout, Conv1D, GlobalMaxPooling1D, Concatenate, Activation
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel, GPT2Config
import sklearn as sk
import os
from nltk.data import find
import matplotlib.pyplot as plt
import re
import tensorflow as tf

In [5]:
train_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:80%]', 'test[80%:]'),
    as_supervised=True)

train_examples, train_labels = next(iter(train_data.batch(20000)))
val_examples, val_labels = next(iter(train_data.batch(5000)))
test_examples, test_labels = next(iter(test_data.batch(1000)))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePYAU5S/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePYAU5S/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePYAU5S/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [37]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

config = GPT2Config.from_pretrained("gpt2", output_hidden_states=True)
gpt2_model = TFGPT2LMHeadModel.from_pretrained("gpt2", config=config)
tokenizer.pad_token = tokenizer.eos_token
MAX_SEQUENCE_LENGTH = 100

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [7]:
#Embedding size of GPT2 tokenizer: 768
#Dictionary size of GPT2 tokenizer: 50257


train_examples_str = [x.decode('utf-8') for x in train_examples.numpy()]
val_examples_str = [x.decode('utf-8') for x in val_examples.numpy()]
test_examples_str = [x.decode('utf-8') for x in test_examples.numpy()]

#training data
gpt_train_tokenized = tokenizer(train_examples_str,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')

gpt_train_inputs = {
    'input_ids': gpt_train_tokenized['input_ids'],
    'attention_mask': gpt_train_tokenized['attention_mask']
}

gpt_train_labels = np.array(train_labels)


#validation data
gpt_val_tokenized = tokenizer(val_examples_str,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')

gpt_val_inputs = {
    'input_ids': gpt_val_tokenized['input_ids'],
    'attention_mask': gpt_val_tokenized['attention_mask']
}

gpt_val_labels = np.array(val_labels)

#testing data
gpt_test_tokenized = tokenizer(test_examples_str,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')

gpt_test_inputs = {
    'input_ids': gpt_test_tokenized['input_ids'],
    'attention_mask': gpt_test_tokenized['attention_mask']
}

gpt_test_labels = np.array(test_labels)


In [38]:
#24 layers of transformer
#A drop out layer + dense layer with 100 hidden layer size on top + final layer with sigmoid as activation function


def create_gpt_last_model(gpt_model,
                          max_sequence_length=MAX_SEQUENCE_LENGTH,
                          hidden_size = 100,
                          dropout=0.3,
                          learning_rate=0.00005):
    """
    Build a simple classification model with gpt. Use the last token output for classification purposes.
    """

    gpt_model.trainable = True #True

    #input layers of gpt, shape (batch, max_sequence_length), model will be fit with gpt_train_inputs
    input_ids = Input(shape=(max_sequence_length,), dtype=tf.int32, name='input_ids')
    attention_mask = Input(shape=(max_sequence_length,), dtype=tf.int32, name='attention_mask')

    # GPT-2 model
    #model.generate() for iteratively generating (autoregressive)
    #we only do it one time.
    gpt2_outputs = gpt_model(input_ids=input_ids, attention_mask=attention_mask)

    # Use the last hidden state of the last transformer layer for classification, ingore linear layer and softmax layer
    # Select the last token of hidden state
    last_hidden_state_last_token = gpt2_outputs.logits[:, -1, :]

    #Add a dropout layer
    x = Dropout(dropout)(last_hidden_state_last_token)

    #Add a fully connected layer for classification
    x = Dense(hidden_size, activation='relu')(x)

    #Final output layer for classification, assuming it's binary task
    output = Dense(1, activation='sigmoid')(x)


    # Create the model
    classification_model = Model(inputs=[input_ids, attention_mask], outputs=output)


    #Model complie
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss='binary_crossentropy',
                                 metrics=['accuracy'])

    return classification_model


In [39]:
gpt_logit_model_classification = create_gpt_last_model(gpt2_model)

history = gpt_logit_model_classification.fit(gpt_train_inputs,
                    gpt_train_labels,
                    epochs=2, #2
                    batch_size=8,
                    validation_data=(gpt_val_inputs, gpt_val_labels))

Epoch 1/2
2500/2500 [==============================] - 788s 301ms/step - loss: 1.9140 - accuracy: 0.6554 - val_loss: 1.6298 - val_accuracy: 0.5326
Epoch 2/2
2500/2500 [==============================] - 727s 291ms/step - loss: 0.6847 - accuracy: 0.5228 - val_loss: 0.6932 - val_accuracy: 0.4948


In [40]:
#Model before zeroing out
gpt_logit_model_classification.evaluate(gpt_test_inputs, gpt_test_labels)

32/32 [==============================] - 17s 372ms/step - loss: 0.6932 - accuracy: 0.4730


[0.6931841969490051, 0.4729999899864197]

In [ ]:
import time

start_time = time.time()
prediction = gpt_logit_model_classification.predict(gpt_test_inputs)
end_time = time.time()

elapsed_time = end_time - start_time
print("Elapsed time: {:.2f} seconds".format(elapsed_time))

In [42]:
"""
Cosine similiarity >　0.35

=================Input 0====================
Layer 10, Neurons: ['Layer 10 Neuron 804']
Layer 11, Neurons: ['Layer 11 Neuron 309', 'Layer 11 Neuron 458', 'Layer 11 Neuron 598', 'Layer 11 Neuron 1120', 'Layer 11 Neuron 1126', 'Layer 11 Neuron 1289', 'Layer 11 Neuron 1322', 'Layer 11 Neuron 1637', 'Layer 11 Neuron 1860']

=================Input 1====================
Layer 10, Neurons: ['Layer 10 Neuron 804', 'Layer 10 Neuron 1410', 'Layer 10 Neuron 1521', 'Layer 10 Neuron 1783', 'Layer 10 Neuron 2931']
Layer 11, Neurons: ['Layer 11 Neuron 309', 'Layer 11 Neuron 458', 'Layer 11 Neuron 480', 'Layer 11 Neuron 598', 'Layer 11 Neuron 796', 'Layer 11 Neuron 847', 'Layer 11 Neuron 869', 'Layer 11 Neuron 975', 'Layer 11 Neuron 1120', 'Layer 11 Neuron 1126',
                    'Layer 11 Neuron 1231', 'Layer 11 Neuron 1289', 'Layer 11 Neuron 1322', 'Layer 11 Neuron 1428', 'Layer 11 Neuron 1575', 'Layer 11 Neuron 1637', 'Layer 11 Neuron 1860', 'Layer 11 Neuron 1998', 'Layer 11 Neuron 2375',
                    'Layer 11 Neuron 2378', 'Layer 11 Neuron 2600', 'Layer 11 Neuron 2822']

=================Input 2====================
Layer 10, Neurons: ['Layer 10 Neuron 804']
Layer 11, Neurons: ['Layer 11 Neuron 309', 'Layer 11 Neuron 458', 'Layer 11 Neuron 598', 'Layer 11 Neuron 1120', 'Layer 11 Neuron 1126', 'Layer 11 Neuron 1289', 'Layer 11 Neuron 1322', 'Layer 11 Neuron 1637', 'Layer 11 Neuron 1860']

=================Input 3====================
Layer 10, Neurons: ['Layer 10 Neuron 804', 'Layer 10 Neuron 1410', 'Layer 10 Neuron 1521', 'Layer 10 Neuron 1783', 'Layer 10 Neuron 2931']
Layer 11, Neurons: ['Layer 11 Neuron 309', 'Layer 11 Neuron 458', 'Layer 11 Neuron 480', 'Layer 11 Neuron 598', 'Layer 11 Neuron 796', 'Layer 11 Neuron 847', 'Layer 11 Neuron 869', 'Layer 11 Neuron 975', 'Layer 11 Neuron 1120', 'Layer 11 Neuron 1126',
                    'Layer 11 Neuron 1231', 'Layer 11 Neuron 1289', 'Layer 11 Neuron 1322', 'Layer 11 Neuron 1428', 'Layer 11 Neuron 1575', 'Layer 11 Neuron 1637', 'Layer 11 Neuron 1860', 'Layer 11 Neuron 1998', 'Layer 11 Neuron 2375',
                    'Layer 11 Neuron 2378', 'Layer 11 Neuron 2600', 'Layer 11 Neuron 2822']

=================Input 4====================
Layer 10, Neurons: ['Layer 10 Neuron 804', 'Layer 10 Neuron 1410']
Layer 11, Neurons: ['Layer 11 Neuron 309', 'Layer 11 Neuron 458', 'Layer 11 Neuron 480', 'Layer 11 Neuron 598', 'Layer 11 Neuron 796', 'Layer 11 Neuron 847', 'Layer 11 Neuron 1120', 'Layer 11 Neuron 1126', 'Layer 11 Neuron 1289', 'Layer 11 Neuron 1322',
                    'Layer 11 Neuron 1637', 'Layer 11 Neuron 1860', 'Layer 11 Neuron 2375', 'Layer 11 Neuron 2378']

=================Input 5====================
Layer 10, Neurons: ['Layer 10 Neuron 804']
Layer 11, Neurons: ['Layer 11 Neuron 309', 'Layer 11 Neuron 458', 'Layer 11 Neuron 1126', 'Layer 11 Neuron 1289', 'Layer 11 Neuron 1637', 'Layer 11 Neuron 1860']

=================Input 6====================
Layer 10, Neurons: ['Layer 10 Neuron 804', 'Layer 10 Neuron 1410']
Layer 11, Neurons: ['Layer 11 Neuron 309', 'Layer 11 Neuron 458', 'Layer 11 Neuron 598', 'Layer 11 Neuron 1120', 'Layer 11 Neuron 1126', 'Layer 11 Neuron 1289', 'Layer 11 Neuron 1322', 'Layer 11 Neuron 1637', 'Layer 11 Neuron 1860', 'Layer 11 Neuron 2375',
                    'Layer 11 Neuron 2378', 'Layer 11 Neuron 2822']

=================Input 7====================
Layer 10, Neurons: ['Layer 10 Neuron 804', 'Layer 10 Neuron 1410', 'Layer 10 Neuron 1783']
Layer 11, Neurons: ['Layer 11 Neuron 309', 'Layer 11 Neuron 458', 'Layer 11 Neuron 480', 'Layer 11 Neuron 598', 'Layer 11 Neuron 796', 'Layer 11 Neuron 847', 'Layer 11 Neuron 975', 'Layer 11 Neuron 1120', 'Layer 11 Neuron 1126', 'Layer 11 Neuron 1231',
                    'Layer 11 Neuron 1289', 'Layer 11 Neuron 1322', 'Layer 11 Neuron 1428', 'Layer 11 Neuron 1637', 'Layer 11 Neuron 1860', 'Layer 11 Neuron 2375', 'Layer 11 Neuron 2378', 'Layer 11 Neuron 2822']


=================Input 8====================
Layer 10, Neurons: ['Layer 10 Neuron 804']
Layer 11, Neurons: ['Layer 11 Neuron 309', 'Layer 11 Neuron 458', 'Layer 11 Neuron 598', 'Layer 11 Neuron 1120', 'Layer 11 Neuron 1126', 'Layer 11 Neuron 1289', 'Layer 11 Neuron 1322', 'Layer 11 Neuron 1637', 'Layer 11 Neuron 1860']

=================Input 9====================
Layer 10, Neurons: ['Layer 10 Neuron 804', 'Layer 10 Neuron 1410']
Layer 11, Neurons: ['Layer 11 Neuron 309', 'Layer 11 Neuron 458', 'Layer 11 Neuron 598', 'Layer 11 Neuron 1120', 'Layer 11 Neuron 1126', 'Layer 11 Neuron 1289', 'Layer 11 Neuron 1322', 'Layer 11 Neuron 1637', 'Layer 11 Neuron 1860', 'Layer 11 Neuron 2375', 'Layer 11 Neuron 2378']
"""




#Create masks for each inputs and zero out the certain portion of neurons randomly except preserved neruons.
#Mask after second layer of FFN
preserved_neuron_list = [[],
                       [],
                       [],
                       [],
                       [],
                       [],
                       [],
                       [],
                       [],
                       [],
                       [804, 1410, 1521, 1783, 2931],
                       [309, 458, 480, 598, 796, 847, 869, 975, 1120, 1126, 1231, 1289, 1322, 1428, 1575, 1637, 1860, 1998, 2375, 2378, 2600, 2822]]

num_neurons = 3072
masks = []

original_list = list(range(3072))
np.random.shuffle(original_list, )
masked_neurons_list = [original_list[:2150]]*12

for i, masked_neurons in enumerate(masked_neurons_list):
    mask = np.ones(num_neurons)
    if masked_neurons not in preserved_neuron_list[i]:
      mask[masked_neurons] = 0
      masks.append(mask)

In [13]:
layer = 'tfgpt2lm_head_model/transformer/h_._0/mlp/c_proj/weight:0'
layer.split('/')[2].split('_')

['h', '.', '0']

In [43]:
#zero out -> quantization + sparse matrix pruning
# Assuming bert_model is your pre-trained BERT model
#tf_bert_model/bert/encoder/layer_._0/output/dense/kernel
#  f"tfgpt2lm_head_model/transformer/h_._{layer_num}/mlp/c_proj/weight:0"


for var in gpt2_model.variables:
    if 'mlp/c_proj/weight' in var.name:
        # Extract layer number from variable name
        layer_num = int(var.name.split('/')[2].split('_')[2])

        # Get the current weights
        weights = var.numpy()

        # Apply the mask #(3072,)
        mask = masks[layer_num]
        weights *= mask.reshape(-1, 1)  # Reshape mask and apply to weights

        # Assign the modified weights back to the variable
        var.assign(weights)


In [15]:
#to check in if the weights are correctly zero out
for var in gpt2_model.variables:
    if 'mlp/c_proj/weight' in var.name:
        print(var.name, var.numpy()[0:3, 0:3])  # Print a small section of the weights

tfgpt2lm_head_model/transformer/h_._0/mlp/c_proj/weight:0 [[-0.1075532   0.15286891  0.03369195]
 [ 0.03780183 -0.05976312  0.08500852]
 [-0.          0.         -0.        ]]
tfgpt2lm_head_model/transformer/h_._1/mlp/c_proj/weight:0 [[ 0.1110587  -0.02688417 -0.03325097]
 [-0.2101619  -0.15602058 -0.08744899]
 [ 0.         -0.          0.        ]]
tfgpt2lm_head_model/transformer/h_._2/mlp/c_proj/weight:0 [[-0.0253068  -0.20976079 -0.02920432]
 [ 0.12399544  0.02172614 -0.11810844]
 [-0.         -0.         -0.        ]]
tfgpt2lm_head_model/transformer/h_._3/mlp/c_proj/weight:0 [[ 0.07656103  0.00484585 -0.08212704]
 [ 0.02443175 -0.00285275  0.0400161 ]
 [ 0.          0.          0.        ]]
tfgpt2lm_head_model/transformer/h_._4/mlp/c_proj/weight:0 [[-0.01662196  0.03226025 -0.02653241]
 [-0.08014955  0.01428386  0.13409194]
 [-0.          0.          0.        ]]
tfgpt2lm_head_model/transformer/h_._5/mlp/c_proj/weight:0 [[ 0.06295791  0.1057768   0.0997331 ]
 [-0.16632158  0.110401

In [34]:
#bert_model has been zeroed out.
gpt_logit_model_classification.evaluate(gpt_test_inputs, gpt_test_labels)

#80~84
#  ~27.39

#            accuracy process time (1000 inputs)
# 0%        --  0.844 27.39
#10% (307)  --  0.827 11.93
#30% (921)  --  0.657 11.81
#50% (1531) --  0.582 12.59
#70% (2150) --
#80% (2457) --
#90% (2764) --
#95% (2922) --

#Impending for further verifying

32/32 [==============================] - 12s 381ms/step - loss: 0.6604 - accuracy: 0.6570


[0.6604329347610474, 0.6570000052452087]

In [36]:
import time

start_time = time.time()
prediction = gpt_logit_model_classification.predict(gpt_test_inputs)
end_time = time.time()

elapsed_time = end_time - start_time
print("Elapsed time: {:.2f} seconds".format(elapsed_time))

32/32 [==============================] - 12s 376ms/step
Elapsed time: 11.81 seconds
